# Imputación de Datos con KNNImputer

In [1]:
import pandas as pd
from sklearn.impute import KNNImputer

from ind import municipality_profile

Lectura del dataset

In [2]:
df = pd.read_csv('../data/processed/dataset_municipal_v2.csv')
df.head()

,cve_mun,municipio,pob_total,pob_indigena,pob_afrodescendiente,viv,viv_internet,viv_computadoras,viv_celular,ind_rez_social,...,licenciatura_percentage,especialidad_percentage,maestr_a_percentage,doctorado_percentage,indice_carencia_compuesto,pct_hacinamiento,pct_agua_entubada,pct_drenaje,pct_piso_tierra,pct_uso_lena
0,20001,Abejones,841,815,1,217,17,17,92,0.815821,...,12.646612,0.173986,0.79035,0.116028,19.447005,3.23,0.46,23.04,70.51,0.0
1,20002,Acatlán de Pérez Figueroa,45167,8548,316,12859,3700,1715,10141,0.066739,...,12.646612,0.173986,0.79035,0.116028,19.709153,8.72,5.52,5.44,78.87,0.0
2,20003,Asunción Cacalotepec,2547,2504,0,794,43,54,202,1.873466,...,12.646612,0.173986,0.79035,0.116028,31.435768,3.65,8.31,75.06,70.15,0.0
3,20004,Asunción Cuyotepeji,1107,3,15,342,60,47,118,-0.158009,...,12.646612,0.173986,0.79035,0.116028,19.532164,5.85,1.46,10.23,80.12,0.0
4,20005,Asunción Ixtaltepec,15261,10406,723,4828,1409,1013,3733,-0.494904,...,12.646612,0.173986,0.79035,0.116028,16.872411,6.30,1.82,2.55,73.70,0.0


Columnas con datos faltantes

In [3]:
to_keep = ["gini", "tasa_ingresos", "total_elementos_salud"]
all_cols_missing = []

# show count of missing values per column
for col in df.columns:
    missing_count = df[col].isnull().sum()
    if missing_count > 0:
        print(f"{col}: {missing_count} missing values")
        all_cols_missing.append(col)

# remove columns with missing values except those in to_keep
to_remove = [col for col in all_cols_missing if col not in to_keep]
print(f"\nRemoving ({len(to_remove)}) columns: {to_remove}")
df_cleaned = df.drop(columns=to_remove)
df_cleaned.head()

gini: 4 missing values
tasa_ingresos: 10 missing values
cama_hospital: 534 missing values
cama_fuera_hospital: 475 missing values
cama_rec_nac: 537 missing values
consultorios: 167 missing values
equipamento: 510 missing values
medicos_otros: 531 missing values
medicos_gen_esp_odont: 199 missing values
otro_pers: 455 missing values
otro_pers_prof: 446 missing values
enfermeria_contac_paciente: 180 missing values
enfermeria_otras: 530 missing values
medicos_adiest: 319 missing values
per_tecnico: 421 missing values
total_elementos_salud: 167 missing values

Removing (13) columns: ['cama_hospital', 'cama_fuera_hospital', 'cama_rec_nac', 'consultorios', 'equipamento', 'medicos_otros', 'medicos_gen_esp_odont', 'otro_pers', 'otro_pers_prof', 'enfermeria_contac_paciente', 'enfermeria_otras', 'medicos_adiest', 'per_tecnico']


,cve_mun,municipio,pob_total,pob_indigena,pob_afrodescendiente,viv,viv_internet,viv_computadoras,viv_celular,ind_rez_social,...,licenciatura_percentage,especialidad_percentage,maestr_a_percentage,doctorado_percentage,indice_carencia_compuesto,pct_hacinamiento,pct_agua_entubada,pct_drenaje,pct_piso_tierra,pct_uso_lena
0,20001,Abejones,841,815,1,217,17,17,92,0.815821,...,12.646612,0.173986,0.79035,0.116028,19.447005,3.23,0.46,23.04,70.51,0.0
1,20002,Acatlán de Pérez Figueroa,45167,8548,316,12859,3700,1715,10141,0.066739,...,12.646612,0.173986,0.79035,0.116028,19.709153,8.72,5.52,5.44,78.87,0.0
2,20003,Asunción Cacalotepec,2547,2504,0,794,43,54,202,1.873466,...,12.646612,0.173986,0.79035,0.116028,31.435768,3.65,8.31,75.06,70.15,0.0
3,20004,Asunción Cuyotepeji,1107,3,15,342,60,47,118,-0.158009,...,12.646612,0.173986,0.79035,0.116028,19.532164,5.85,1.46,10.23,80.12,0.0
4,20005,Asunción Ixtaltepec,15261,10406,723,4828,1409,1013,3733,-0.494904,...,12.646612,0.173986,0.79035,0.116028,16.872411,6.30,1.82,2.55,73.70,0.0


Mostrar perfil del municipio

In [4]:
muns = ["Matías Romero Avendaño", "Magdalena Mixtepec", "Asunción Cuyotepeji"]

In [5]:
for municipality_name in muns:
    print(f"\n\nPerfil del municipio antes de la imputación de datos:")
    _ = municipality_profile(df_cleaned, municipality_name)



Perfil del municipio antes de la imputación de datos:
Elementos por mil habitantes: 3.7713118403477988
Total elementos salud: 144.0, Población total: 38183

================= PERFIL DEL MUNICIPIO: MATÍAS ROMERO AVENDAÑO =================

RESUMEN DEL PERFIL:
  • Marginación: Medio
  • Rezago social: Bajo
  • Conectividad: Conectividad media
  • Servicios básicos: Bajo acceso a servicios básicos
  • Salud: Acceso moderado a servicios de salud
  • Educación: Rezago educativo moderado
  • Desigualdad: Sin datos
  • Dependencia económica: Dependencia económica moderada


Perfil del municipio antes de la imputación de datos:
Elementos por mil habitantes: 2.7913468248429867
Total elementos salud: 4.0, Población total: 1433

=================== PERFIL DEL MUNICIPIO: MAGDALENA MIXTEPEC ===================

RESUMEN DEL PERFIL:
  • Marginación: Alto
  • Rezago social: Muy alto
  • Conectividad: Baja conectividad
  • Servicios básicos: Muy bajo acceso a servicios básicos
  • Salud: Acceso moder

Quitar cve_mun y municipio

In [6]:
metadata = ["cve_mun", "municipio"]

df_cleaned = df_cleaned.drop(columns=metadata)
df_cleaned.head()

,pob_total,pob_indigena,pob_afrodescendiente,viv,viv_internet,viv_computadoras,viv_celular,ind_rez_social,grad_rez_social,ind_margi,...,licenciatura_percentage,especialidad_percentage,maestr_a_percentage,doctorado_percentage,indice_carencia_compuesto,pct_hacinamiento,pct_agua_entubada,pct_drenaje,pct_piso_tierra,pct_uso_lena
0,841,815,1,217,17,17,92,0.815821,Medio,52.051931,...,12.646612,0.173986,0.79035,0.116028,19.447005,3.23,0.46,23.04,70.51,0.0
1,45167,8548,316,12859,3700,1715,10141,0.066739,Bajo,52.951754,...,12.646612,0.173986,0.79035,0.116028,19.709153,8.72,5.52,5.44,78.87,0.0
2,2547,2504,0,794,43,54,202,1.873466,Muy alto,49.945211,...,12.646612,0.173986,0.79035,0.116028,31.435768,3.65,8.31,75.06,70.15,0.0
3,1107,3,15,342,60,47,118,-0.158009,Bajo,54.572186,...,12.646612,0.173986,0.79035,0.116028,19.532164,5.85,1.46,10.23,80.12,0.0
4,15261,10406,723,4828,1409,1013,3733,-0.494904,Bajo,56.007167,...,12.646612,0.173986,0.79035,0.116028,16.872411,6.30,1.82,2.55,73.70,0.0


Ejemplos de valores faltantes para las columnas seleccionadas

In [7]:
# for each column in to_keep, show the first 3 rows with missing values
to_show = []
for col in to_keep:
    missing_rows = df_cleaned[df_cleaned[col].isnull()].head(3)
    if not missing_rows.empty:
        print(f"\nPrimeras 3 instancias con valores vacíos en '{col}':")
        # print(missing_rows[["municipio", col]])

        # imprima el nombre del municipio en df y el valor faltante
        for index, row in missing_rows.iterrows():
            print(f"Municipio: {df.loc[index, 'municipio']}, Valor faltante en '{col}': {row[col]}")

        to_show.append({
            "column": col,
            "rows": missing_rows.index
        })


Primeras 3 instancias con valores vacíos en 'gini':
Municipio: Matías Romero Avendaño, Valor faltante en 'gini': nan
Municipio: San Francisco Chindúa, Valor faltante en 'gini': nan
Municipio: Santa María Chimalapa, Valor faltante en 'gini': nan

Primeras 3 instancias con valores vacíos en 'tasa_ingresos':
Municipio: Magdalena Mixtepec, Valor faltante en 'tasa_ingresos': nan
Municipio: San Andrés Tepetlapa, Valor faltante en 'tasa_ingresos': nan
Municipio: San Francisco Logueche, Valor faltante en 'tasa_ingresos': nan

Primeras 3 instancias con valores vacíos en 'total_elementos_salud':
Municipio: Asunción Cuyotepeji, Valor faltante en 'total_elementos_salud': nan
Municipio: Calihualá, Valor faltante en 'total_elementos_salud': nan
Municipio: Constancia del Rosario, Valor faltante en 'total_elementos_salud': nan


Imputación de los datos

In [8]:
# imput with KNNImputer
sub_df = df_cleaned[to_keep]
imputer = KNNImputer(n_neighbors=3)
imputed = imputer.fit_transform(sub_df)

# Re-add to df_cleaned
for i, col in enumerate(to_keep):
    df_cleaned[col] = imputed[:, i]

# Ceil round for 'total_elementos_salud'
df_cleaned['total_elementos_salud'] = df_cleaned['total_elementos_salud'].round().astype(int)

Agregar columnas de metadatos

In [9]:
# add back metadata columns
df_final = pd.concat([df[metadata], df_cleaned], axis=1)

Mostrar los valores imputados

In [ ]:
for item in to_show:
    col = item["column"]
    rows = item["rows"]
    print(f"\nValores imputados para '{col}':")
    print(df_final.loc[rows, ["municipio", col]])


Valores imputados para 'gini':
                  municipio      gini
56   Matías Romero Avendaño  0.389692
139   San Francisco Chindúa  0.369091
406   Santa María Chimalapa  0.381204

Valores imputados para 'tasa_ingresos':
                  municipio  tasa_ingresos
47       Magdalena Mixtepec       0.195435
98     San Andrés Tepetlapa       0.167785
145  San Francisco Logueche       0.204266

Valores imputados para 'total_elementos_salud':
                 municipio  total_elementos_salud
3      Asunción Cuyotepeji                      9
10               Calihualá                     12
19  Constancia del Rosario                     70


Mostrar perfil de los municipios después de la imputación de datos

In [11]:
# show profile after imputation
for municipality_name in muns:
    print(f"\n\nPerfil del municipio después de la imputación de datos:")
    _ = municipality_profile(df_final, municipality_name)



Perfil del municipio después de la imputación de datos:
Elementos por mil habitantes: 3.7713118403477988
Total elementos salud: 144, Población total: 38183

================= PERFIL DEL MUNICIPIO: MATÍAS ROMERO AVENDAÑO =================

RESUMEN DEL PERFIL:
  • Marginación: Medio
  • Rezago social: Bajo
  • Conectividad: Conectividad media
  • Servicios básicos: Bajo acceso a servicios básicos
  • Salud: Acceso moderado a servicios de salud
  • Educación: Rezago educativo moderado
  • Desigualdad: Desigualdad moderada
  • Dependencia económica: Dependencia económica moderada


Perfil del municipio después de la imputación de datos:
Elementos por mil habitantes: 2.7913468248429867
Total elementos salud: 4, Población total: 1433

=================== PERFIL DEL MUNICIPIO: MAGDALENA MIXTEPEC ===================

RESUMEN DEL PERFIL:
  • Marginación: Alto
  • Rezago social: Muy alto
  • Conectividad: Baja conectividad
  • Servicios básicos: Muy bajo acceso a servicios básicos
  • Salud: A